In [ ]:
{
 "metadata": {
  "name": ""
 },
 "nbformat": 3,
 "nbformat_minor": 0,
 "worksheets": [
  {
   "cells": [
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "<div style=\"float: right; padding: 20px 0px 0px 20px;\"><img src=\"files/images/dr_perceptron.jpg\" width=\"300px\"></div>\n",
      "## PE File Classification Exercise\n",
      "<font size=4>In this notebook we're going to explore, understand and classify PE (Portable Executable) files as being 'benign' or 'malicious'.\n",
      "http://en.wikipedia.org/wiki/Portable_Executable\n",
      "The primary motivation is to explore the nexus of IPython, Pandas and scikit-learn with PE File classification as a vehicle for that exploration. The exercise intentionally shows what machine learning experts might call a naive approach, this is for clarity and conciseness. Recommendations for deeper materials and resources are given in the conclusion.\n",
      "<br><br>\n",
      "** DISCLAIMER:** This exercise is for illustrative purposes and only uses about 100 samples which is way too small for a generalizable model.\n",
      "</font>\n",
      "### Python Modules Used:\n",
      "<font size=4>\n",
      "<ul>\n",
      "<li>Pandas: Python Data Analysis Library (http://pandas.pydata.org)\n",
      "<li>Scikit Learn (http://scikit-learn.org) Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.\n",
      "<li>Matplotlib: Python 2D plotting library (http://matplotlib.org)\n",
      "</ul>\n",
      "<br>\n",
      "**All Code and IPython Notebooks for this talk: http://clicksecurity.github.io/data_hacking**\n",
      "</font>\n",
      "<br>"
     ]
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "### Imports and plot defaults"
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "import os\n",
      "import sklearn.feature_extraction\n",
      "sklearn.__version__"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 190,
       "text": [
        "'0.14.1'"
       ]
      }
     ],
     "prompt_number": 190
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "import pandas as pd\n",
      "pd.__version__"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 191,
       "text": [
        "'0.13.1'"
       ]
      }
     ],
     "prompt_number": 191
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "import numpy as np\n",
      "np.__version__"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 192,
       "text": [
        "'1.8.0'"
       ]
      }
     ],
     "prompt_number": 192
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Plotting defaults\n",
      "import matplotlib.pyplot as plt\n",
      "%matplotlib inline\n",
      "plt.rcParams['font.size'] = 18.0\n",
      "plt.rcParams['figure.figsize'] = 16.0, 5.0"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [],
     "prompt_number": 193
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "def plot_cm(cm, labels):\n",
      "    # Compute percentanges\n",
      "    percent = (cm*100.0)/np.array(np.matrix(cm.sum(axis=1)).T)  # Derp, I'm sure there's a better way   \n",
      "    print 'Confusion Matrix Stats'\n",
      "    for i, label_i in enumerate(labels):\n",
      "        for j, label_j in enumerate(labels):\n",
      "            print \"%s/%s: %.2f%% (%d/%d)\" % (label_i, label_j, (percent[i][j]), cm[i][j], cm[i].sum())\n",
      "\n",
      "    # Show confusion matrix\n",
      "    # Thanks kermit666 from stackoverflow :)\n",
      "    fig = plt.figure()\n",
      "    ax = fig.add_subplot(111)\n",
      "    ax.grid(b=False)\n",
      "    cax = ax.matshow(percent, cmap='coolwarm',vmin=0,vmax=100)\n",
      "    plt.title('Confusion matrix of the classifier')\n",
      "    fig.colorbar(cax)\n",
      "    ax.set_xticklabels([''] + labels)\n",
      "    ax.set_yticklabels([''] + labels)\n",
      "    plt.xlabel('Predicted')\n",
      "    plt.ylabel('True')\n",
      "    plt.show()"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "skip"
      }
     },
     "outputs": [],
     "prompt_number": 194
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "import os, warnings\n",
      "warnings.filterwarnings(\"ignore\", category=DeprecationWarning)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "skip"
      }
     },
     "outputs": [],
     "prompt_number": 195
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "<div style=\"float: left; margin: 0px 30px 0px 0px\"><img src=\"files/images/disk.jpg\" width=\"300px\"></div>\n",
      "# Read in the Raw Data\n",
      "<font size=4>For PE files we want to quickly go from the raw binary files to a feature vector (DataFrame). For PE files there are lots of great tools, there's the pefile python module written by Ero Carrera and there's also a nice new github project called PEFrame  https://github.com/guelfoweb/peframe by Gianni Amato at http://www.securityside.it. For this exercise we've provided a little wrapper class around the pefile module.</font>"
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "import pe_features\n",
      "my_extractor = pe_features.PEFileFeatures()\n",
      "\n",
      "# Open a PE File and see what features we get\n",
      "filename = 'data/bad/0cb9aa6fb9c4aa3afad7a303e21ac0f3'\n",
      "with open(filename,'rb') as f:\n",
      "    features = my_extractor.execute(f.read())\n",
      "features"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 196,
       "text": [
        "{'check_sum': 0,\n",
        " 'compile_date': 1218437803,\n",
        " 'datadir_IMAGE_DIRECTORY_ENTRY_BASERELOC_size': 0,\n",
        " 'datadir_IMAGE_DIRECTORY_ENTRY_EXPORT_size': 0,\n",
        " 'datadir_IMAGE_DIRECTORY_ENTRY_IAT_size': 468,\n",
        " 'datadir_IMAGE_DIRECTORY_ENTRY_IMPORT_size': 100,\n",
        " 'datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size': 1048,\n",
        " 'debug_size': 0,\n",
        " 'export_size': 0,\n",
        " 'generated_check_sum': 53913,\n",
        " 'iat_rva': 9256,\n",
        " 'major_version': 0,\n",
        " 'minor_version': 0,\n",
        " 'number_of_bound_import_symbols': 0,\n",
        " 'number_of_bound_imports': 0,\n",
        " 'number_of_export_symbols': 0,\n",
        " 'number_of_import_symbols': 113,\n",
        " 'number_of_imports': 4,\n",
        " 'number_of_rva_and_sizes': 16,\n",
        " 'number_of_sections': 4,\n",
        " 'pe_char': 271,\n",
        " 'pe_dll': 0,\n",
        " 'pe_driver': 0,\n",
        " 'pe_exe': 1,\n",
        " 'pe_i386': 1,\n",
        " 'pe_majorlink': 6,\n",
        " 'pe_minorlink': 0,\n",
        " 'pe_warnings': 0,\n",
        " 'sec_entropy_data': 0.4421475832668401,\n",
        " 'sec_entropy_rdata': 3.2064873564662046,\n",
        " 'sec_entropy_reloc': 0,\n",
        " 'sec_entropy_rsrc': 1.028676764457129,\n",
        " 'sec_entropy_text': 4.852962403013336,\n",
        " 'sec_raw_execsize': 16384,\n",
        " 'sec_rawptr_data': 12288,\n",
        " u'sec_rawptr_rdata': 8192,\n",
        " 'sec_rawptr_rsrc': 16384,\n",
        " 'sec_rawptr_text': 4096,\n",
        " 'sec_rawsize_data': 4096,\n",
        " u'sec_rawsize_rdata': 4096,\n",
        " 'sec_rawsize_rsrc': 4096,\n",
        " 'sec_rawsize_text': 4096,\n",
        " 'sec_va_execsize': 7044,\n",
        " 'sec_vasize_data': 468,\n",
        " u'sec_vasize_rdata': 2182,\n",
        " 'sec_vasize_rsrc': 1048,\n",
        " 'sec_vasize_text': 3346,\n",
        " 'size_code': 4096,\n",
        " 'size_image': 20480,\n",
        " 'size_initdata': 12288,\n",
        " 'size_uninit': 0,\n",
        " 'std_section_names': 1,\n",
        " 'total_size_pe': 20480,\n",
        " 'virtual_address': 4096,\n",
        " 'virtual_size': 3346,\n",
        " 'virtual_size_2': 2182}"
       ]
      }
     ],
     "prompt_number": 196
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Load up all our files (files come from various places contagio, around the net...)\n",
      "def load_files(file_list):\n",
      "    features_list = []\n",
      "    for filename in file_list:\n",
      "        with open(filename,'rb') as f:\n",
      "            features_list.append(my_extractor.execute(f.read()))\n",
      "    return features_list"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [],
     "prompt_number": 197
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Bad (malicious) files\n",
      "file_list = [os.path.join('data/bad', child) for child in os.listdir('data/bad')]\n",
      "bad_features = load_files(file_list)\n",
      "print 'Loaded up %d malicious PE Files' % len(bad_features)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Loaded up 50 malicious PE Files\n"
       ]
      }
     ],
     "prompt_number": 198
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Good (benign) files\n",
      "file_list = [os.path.join('data/good', child) for child in os.listdir('data/good')]\n",
      "good_features = load_files(file_list)\n",
      "print 'Loaded up %d benign PE Files' % len(good_features)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Loaded up 50 benign PE Files\n"
       ]
      }
     ],
     "prompt_number": 199
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "<div style=\"float: left; padding: 0px 20px 0px 20px;\"><img src=\"files/images/transformers.png\" width=\"300px\"></div>\n",
      "# Data Transformation: \n",
      "** Going from a list of python dictionaries to a Pandas DataFrame. Pandas has all sort of different ways to create a data frame. **"
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Putting the features into a pandas dataframe\n",
      "import pandas as pd\n",
      "df_bad = pd.DataFrame.from_records(bad_features)\n",
      "df_bad['label'] = 'bad'\n",
      "df_good = pd.DataFrame.from_records(good_features)\n",
      "df_good['label'] = 'good'\n",
      "df_good.head()"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "html": [
        "<div style=\"max-height:1000px;max-width:1500px;overflow:auto;\">\n",
        "<table border=\"1\" class=\"dataframe\">\n",
        "  <thead>\n",
        "    <tr style=\"text-align: right;\">\n",
        "      <th></th>\n",
        "      <th>check_sum</th>\n",
        "      <th>compile_date</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_BASERELOC_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_EXPORT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_IAT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_IMPORT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size</th>\n",
        "      <th>debug_size</th>\n",
        "      <th>export_size</th>\n",
        "      <th>generated_check_sum</th>\n",
        "      <th>iat_rva</th>\n",
        "      <th>major_version</th>\n",
        "      <th>minor_version</th>\n",
        "      <th>number_of_bound_import_symbols</th>\n",
        "      <th>number_of_bound_imports</th>\n",
        "      <th>number_of_export_symbols</th>\n",
        "      <th>number_of_import_symbols</th>\n",
        "      <th>number_of_imports</th>\n",
        "      <th>number_of_rva_and_sizes</th>\n",
        "      <th>number_of_sections</th>\n",
        "      <th></th>\n",
        "    </tr>\n",
        "  </thead>\n",
        "  <tbody>\n",
        "    <tr>\n",
        "      <th>0</th>\n",
        "      <td>  97308</td>\n",
        "      <td> 1383744221</td>\n",
        "      <td> 3044</td>\n",
        "      <td> 0</td>\n",
        "      <td>  592</td>\n",
        "      <td> 140</td>\n",
        "      <td>  7368</td>\n",
        "      <td> 28</td>\n",
        "      <td> 0</td>\n",
        "      <td>  97308</td>\n",
        "      <td> 50424</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 142</td>\n",
        "      <td> 6</td>\n",
        "      <td> 16</td>\n",
        "      <td> 5</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>1</th>\n",
        "      <td> 103233</td>\n",
        "      <td> 1383102953</td>\n",
        "      <td>   60</td>\n",
        "      <td> 0</td>\n",
        "      <td> 1008</td>\n",
        "      <td>  60</td>\n",
        "      <td>   872</td>\n",
        "      <td> 28</td>\n",
        "      <td> 0</td>\n",
        "      <td> 103233</td>\n",
        "      <td> 53248</td>\n",
        "      <td> 5</td>\n",
        "      <td> 1</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 124</td>\n",
        "      <td> 2</td>\n",
        "      <td> 16</td>\n",
        "      <td> 8</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>2</th>\n",
        "      <td>  26573</td>\n",
        "      <td> 1386271379</td>\n",
        "      <td>  360</td>\n",
        "      <td> 0</td>\n",
        "      <td>  208</td>\n",
        "      <td> 100</td>\n",
        "      <td>  2588</td>\n",
        "      <td> 28</td>\n",
        "      <td> 0</td>\n",
        "      <td>  25971</td>\n",
        "      <td>  8804</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td>  48</td>\n",
        "      <td> 4</td>\n",
        "      <td> 16</td>\n",
        "      <td> 5</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>3</th>\n",
        "      <td>      0</td>\n",
        "      <td> 1373925025</td>\n",
        "      <td>   12</td>\n",
        "      <td> 0</td>\n",
        "      <td>    8</td>\n",
        "      <td>  83</td>\n",
        "      <td> 11904</td>\n",
        "      <td> 28</td>\n",
        "      <td> 0</td>\n",
        "      <td>  54015</td>\n",
        "      <td> 35064</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td>   1</td>\n",
        "      <td> 1</td>\n",
        "      <td> 16</td>\n",
        "      <td> 4</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>4</th>\n",
        "      <td>  50003</td>\n",
        "      <td> 1378865704</td>\n",
        "      <td>  360</td>\n",
        "      <td> 0</td>\n",
        "      <td>  208</td>\n",
        "      <td> 100</td>\n",
        "      <td>  2588</td>\n",
        "      <td> 28</td>\n",
        "      <td> 0</td>\n",
        "      <td>  59485</td>\n",
        "      <td>  8804</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td> 0</td>\n",
        "      <td>  48</td>\n",
        "      <td> 4</td>\n",
        "      <td> 16</td>\n",
        "      <td> 5</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "  </tbody>\n",
        "</table>\n",
        "<p>5 rows \u00d7 108 columns</p>\n",
        "</div>"
       ],
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 200,
       "text": [
        "   check_sum  compile_date  datadir_IMAGE_DIRECTORY_ENTRY_BASERELOC_size  \\\n",
        "0      97308    1383744221                                          3044   \n",
        "1     103233    1383102953                                            60   \n",
        "2      26573    1386271379                                           360   \n",
        "3          0    1373925025                                            12   \n",
        "4      50003    1378865704                                           360   \n",
        "\n",
        "   datadir_IMAGE_DIRECTORY_ENTRY_EXPORT_size  \\\n",
        "0                                          0   \n",
        "1                                          0   \n",
        "2                                          0   \n",
        "3                                          0   \n",
        "4                                          0   \n",
        "\n",
        "   datadir_IMAGE_DIRECTORY_ENTRY_IAT_size  \\\n",
        "0                                     592   \n",
        "1                                    1008   \n",
        "2                                     208   \n",
        "3                                       8   \n",
        "4                                     208   \n",
        "\n",
        "   datadir_IMAGE_DIRECTORY_ENTRY_IMPORT_size  \\\n",
        "0                                        140   \n",
        "1                                         60   \n",
        "2                                        100   \n",
        "3                                         83   \n",
        "4                                        100   \n",
        "\n",
        "   datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size  debug_size  export_size  \\\n",
        "0                                         7368          28            0   \n",
        "1                                          872          28            0   \n",
        "2                                         2588          28            0   \n",
        "3                                        11904          28            0   \n",
        "4                                         2588          28            0   \n",
        "\n",
        "   generated_check_sum  iat_rva  major_version  minor_version  \\\n",
        "0                97308    50424              0              0   \n",
        "1               103233    53248              5              1   \n",
        "2                25971     8804              0              0   \n",
        "3                54015    35064              0              0   \n",
        "4                59485     8804              0              0   \n",
        "\n",
        "   number_of_bound_import_symbols  number_of_bound_imports  \\\n",
        "0                               0                        0   \n",
        "1                               0                        0   \n",
        "2                               0                        0   \n",
        "3                               0                        0   \n",
        "4                               0                        0   \n",
        "\n",
        "   number_of_export_symbols  number_of_import_symbols  number_of_imports  \\\n",
        "0                         0                       142                  6   \n",
        "1                         0                       124                  2   \n",
        "2                         0                        48                  4   \n",
        "3                         0                         1                  1   \n",
        "4                         0                        48                  4   \n",
        "\n",
        "   number_of_rva_and_sizes  number_of_sections      \n",
        "0                       16                   5 ...  \n",
        "1                       16                   8 ...  \n",
        "2                       16                   5 ...  \n",
        "3                       16                   4 ...  \n",
        "4                       16                   5 ...  \n",
        "\n",
        "[5 rows x 108 columns]"
       ]
      }
     ],
     "prompt_number": 200
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "<div style=\"float: left; margin: 0px 30px 0px 0px\"><img src=\"files/images/eyeball_2.jpg\" width=\"250px\"></div>\n",
      "# Lets look at the Data\n",
      "We're going to use some nice functionality in the Pandas dataframe to look at our processed data:"
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Now we're set and we open up a a whole new world!\n",
      "\n",
      "# Gisting and statistics\n",
      "df_bad.describe()"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "html": [
        "<div style=\"max-height:1000px;max-width:1500px;overflow:auto;\">\n",
        "<table border=\"1\" class=\"dataframe\">\n",
        "  <thead>\n",
        "    <tr style=\"text-align: right;\">\n",
        "      <th></th>\n",
        "      <th>check_sum</th>\n",
        "      <th>compile_date</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_BASERELOC_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_EXPORT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_IAT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_IMPORT_size</th>\n",
        "      <th>datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size</th>\n",
        "      <th>debug_size</th>\n",
        "      <th>export_size</th>\n",
        "      <th>generated_check_sum</th>\n",
        "      <th>iat_rva</th>\n",
        "      <th>major_version</th>\n",
        "      <th>minor_version</th>\n",
        "      <th>number_of_bound_import_symbols</th>\n",
        "      <th>number_of_bound_imports</th>\n",
        "      <th>number_of_export_symbols</th>\n",
        "      <th>number_of_import_symbols</th>\n",
        "      <th>number_of_imports</th>\n",
        "      <th>number_of_rva_and_sizes</th>\n",
        "      <th>number_of_sections</th>\n",
        "      <th></th>\n",
        "    </tr>\n",
        "  </thead>\n",
        "  <tbody>\n",
        "    <tr>\n",
        "      <th>count</th>\n",
        "      <td>     50.000000</td>\n",
        "      <td> 5.000000e+01</td>\n",
        "      <td>   50.000000</td>\n",
        "      <td>  50.000000</td>\n",
        "      <td>  50.000000</td>\n",
        "      <td>   50.000000</td>\n",
        "      <td>    50.00000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td>  50.000000</td>\n",
        "      <td>     50.000000</td>\n",
        "      <td>     50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td>  50.000000</td>\n",
        "      <td> 50.000000</td>\n",
        "      <td> 50</td>\n",
        "      <td> 50.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>mean</th>\n",
        "      <td>  25235.660000</td>\n",
        "      <td> 1.035770e+09</td>\n",
        "      <td>  415.280000</td>\n",
        "      <td>  14.640000</td>\n",
        "      <td> 126.720000</td>\n",
        "      <td>  456.160000</td>\n",
        "      <td>  9615.64000</td>\n",
        "      <td>  3.920000</td>\n",
        "      <td>  14.640000</td>\n",
        "      <td>  86998.520000</td>\n",
        "      <td>  43982.640000</td>\n",
        "      <td>  0.960000</td>\n",
        "      <td>  0.120000</td>\n",
        "      <td>  0.140000</td>\n",
        "      <td>  0.740000</td>\n",
        "      <td>  0.240000</td>\n",
        "      <td>  44.560000</td>\n",
        "      <td>  3.740000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  4.32000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>std</th>\n",
        "      <td>  45704.015095</td>\n",
        "      <td> 3.202979e+08</td>\n",
        "      <td> 1061.159532</td>\n",
        "      <td>  55.908365</td>\n",
        "      <td> 180.722252</td>\n",
        "      <td> 1060.814846</td>\n",
        "      <td> 19062.02003</td>\n",
        "      <td>  9.814275</td>\n",
        "      <td>  55.908365</td>\n",
        "      <td>  30119.209943</td>\n",
        "      <td>  44546.311213</td>\n",
        "      <td>  2.137708</td>\n",
        "      <td>  0.328261</td>\n",
        "      <td>  0.404566</td>\n",
        "      <td>  2.028471</td>\n",
        "      <td>  0.893514</td>\n",
        "      <td>  46.412595</td>\n",
        "      <td>  3.445257</td>\n",
        "      <td>  0</td>\n",
        "      <td>  1.75476</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>min</th>\n",
        "      <td>      0.000000</td>\n",
        "      <td> 2.099200e+06</td>\n",
        "      <td>    0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>    0.000000</td>\n",
        "      <td>     0.00000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>  26104.000000</td>\n",
        "      <td>      0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  1.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>25%</th>\n",
        "      <td>      0.000000</td>\n",
        "      <td> 9.372855e+08</td>\n",
        "      <td>    0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>   40.000000</td>\n",
        "      <td>     0.00000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>  68094.000000</td>\n",
        "      <td>  14593.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   9.250000</td>\n",
        "      <td>  1.000000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  3.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>50%</th>\n",
        "      <td>      0.000000</td>\n",
        "      <td> 1.172916e+09</td>\n",
        "      <td>    0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>  44.000000</td>\n",
        "      <td>  100.000000</td>\n",
        "      <td>  1152.00000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td>  82579.000000</td>\n",
        "      <td>  25835.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  26.000000</td>\n",
        "      <td>  2.500000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  4.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>75%</th>\n",
        "      <td>  36417.000000</td>\n",
        "      <td> 1.219691e+09</td>\n",
        "      <td>   14.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td> 231.000000</td>\n",
        "      <td>  186.000000</td>\n",
        "      <td>  5938.00000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>   0.000000</td>\n",
        "      <td> 108406.000000</td>\n",
        "      <td>  55948.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  0.000000</td>\n",
        "      <td>  70.500000</td>\n",
        "      <td>  5.000000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  5.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "    <tr>\n",
        "      <th>max</th>\n",
        "      <td> 150326.000000</td>\n",
        "      <td> 1.382647e+09</td>\n",
        "      <td> 4612.000000</td>\n",
        "      <td> 313.000000</td>\n",
        "      <td> 748.000000</td>\n",
        "      <td> 6234.000000</td>\n",
        "      <td> 84152.00000</td>\n",
        "      <td> 28.000000</td>\n",
        "      <td> 313.000000</td>\n",
        "      <td> 164776.000000</td>\n",
        "      <td> 189824.000000</td>\n",
        "      <td> 10.000000</td>\n",
        "      <td>  1.000000</td>\n",
        "      <td>  2.000000</td>\n",
        "      <td>  8.000000</td>\n",
        "      <td>  4.000000</td>\n",
        "      <td> 180.000000</td>\n",
        "      <td> 18.000000</td>\n",
        "      <td> 16</td>\n",
        "      <td>  9.00000</td>\n",
        "      <td>...</td>\n",
        "    </tr>\n",
        "  </tbody>\n",
        "</table>\n",
        "<p>8 rows \u00d7 199 columns</p>\n",
        "</div>"
       ],
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 201,
       "text": [
        "           check_sum  compile_date  \\\n",
        "count      50.000000  5.000000e+01   \n",
        "mean    25235.660000  1.035770e+09   \n",
        "std     45704.015095  3.202979e+08   \n",
        "min         0.000000  2.099200e+06   \n",
        "25%         0.000000  9.372855e+08   \n",
        "50%         0.000000  1.172916e+09   \n",
        "75%     36417.000000  1.219691e+09   \n",
        "max    150326.000000  1.382647e+09   \n",
        "\n",
        "       datadir_IMAGE_DIRECTORY_ENTRY_BASERELOC_size  \\\n",
        "count                                     50.000000   \n",
        "mean                                     415.280000   \n",
        "std                                     1061.159532   \n",
        "min                                        0.000000   \n",
        "25%                                        0.000000   \n",
        "50%                                        0.000000   \n",
        "75%                                       14.000000   \n",
        "max                                     4612.000000   \n",
        "\n",
        "       datadir_IMAGE_DIRECTORY_ENTRY_EXPORT_size  \\\n",
        "count                                  50.000000   \n",
        "mean                                   14.640000   \n",
        "std                                    55.908365   \n",
        "min                                     0.000000   \n",
        "25%                                     0.000000   \n",
        "50%                                     0.000000   \n",
        "75%                                     0.000000   \n",
        "max                                   313.000000   \n",
        "\n",
        "       datadir_IMAGE_DIRECTORY_ENTRY_IAT_size  \\\n",
        "count                               50.000000   \n",
        "mean                               126.720000   \n",
        "std                                180.722252   \n",
        "min                                  0.000000   \n",
        "25%                                  0.000000   \n",
        "50%                                 44.000000   \n",
        "75%                                231.000000   \n",
        "max                                748.000000   \n",
        "\n",
        "       datadir_IMAGE_DIRECTORY_ENTRY_IMPORT_size  \\\n",
        "count                                  50.000000   \n",
        "mean                                  456.160000   \n",
        "std                                  1060.814846   \n",
        "min                                     0.000000   \n",
        "25%                                    40.000000   \n",
        "50%                                   100.000000   \n",
        "75%                                   186.000000   \n",
        "max                                  6234.000000   \n",
        "\n",
        "       datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size  debug_size  export_size  \\\n",
        "count                                     50.00000   50.000000    50.000000   \n",
        "mean                                    9615.64000    3.920000    14.640000   \n",
        "std                                    19062.02003    9.814275    55.908365   \n",
        "min                                        0.00000    0.000000     0.000000   \n",
        "25%                                        0.00000    0.000000     0.000000   \n",
        "50%                                     1152.00000    0.000000     0.000000   \n",
        "75%                                     5938.00000    0.000000     0.000000   \n",
        "max                                    84152.00000   28.000000   313.000000   \n",
        "\n",
        "       generated_check_sum        iat_rva  major_version  minor_version  \\\n",
        "count            50.000000      50.000000      50.000000      50.000000   \n",
        "mean          86998.520000   43982.640000       0.960000       0.120000   \n",
        "std           30119.209943   44546.311213       2.137708       0.328261   \n",
        "min           26104.000000       0.000000       0.000000       0.000000   \n",
        "25%           68094.000000   14593.000000       0.000000       0.000000   \n",
        "50%           82579.000000   25835.000000       0.000000       0.000000   \n",
        "75%          108406.000000   55948.000000       0.000000       0.000000   \n",
        "max          164776.000000  189824.000000      10.000000       1.000000   \n",
        "\n",
        "       number_of_bound_import_symbols  number_of_bound_imports  \\\n",
        "count                       50.000000                50.000000   \n",
        "mean                         0.140000                 0.740000   \n",
        "std                          0.404566                 2.028471   \n",
        "min                          0.000000                 0.000000   \n",
        "25%                          0.000000                 0.000000   \n",
        "50%                          0.000000                 0.000000   \n",
        "75%                          0.000000                 0.000000   \n",
        "max                          2.000000                 8.000000   \n",
        "\n",
        "       number_of_export_symbols  number_of_import_symbols  number_of_imports  \\\n",
        "count                 50.000000                 50.000000          50.000000   \n",
        "mean                   0.240000                 44.560000           3.740000   \n",
        "std                    0.893514                 46.412595           3.445257   \n",
        "min                    0.000000                  0.000000           0.000000   \n",
        "25%                    0.000000                  9.250000           1.000000   \n",
        "50%                    0.000000                 26.000000           2.500000   \n",
        "75%                    0.000000                 70.500000           5.000000   \n",
        "max                    4.000000                180.000000          18.000000   \n",
        "\n",
        "       number_of_rva_and_sizes  number_of_sections      \n",
        "count                       50            50.00000 ...  \n",
        "mean                        16             4.32000 ...  \n",
        "std                          0             1.75476 ...  \n",
        "min                         16             1.00000 ...  \n",
        "25%                         16             3.00000 ...  \n",
        "50%                         16             4.00000 ...  \n",
        "75%                         16             5.00000 ...  \n",
        "max                         16             9.00000 ...  \n",
        "\n",
        "[8 rows x 199 columns]"
       ]
      }
     ],
     "prompt_number": 201
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Visualization I\n",
      "df_bad['check_sum'].hist(alpha=.5,label='bad',bins=40)\n",
      "df_good['check_sum'].hist(alpha=.5,label='good',bins=40)\n",
      "plt.legend()"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 202,
       "text": [
        "<matplotlib.legend.Legend at 0x110fd4b10>"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\nUPp/6Vm98jyROnkAAAAASUVORK5CYII=\n",
       "text": [
        "<matplotlib.figure.Figure at 0x1108d80d0>"
       ]
      }
     ],
     "prompt_number": 202
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Visualization I\n",
      "df_bad['generated_check_sum'].hist(alpha=.5,label='bad',bins=40)\n",
      "df_good['generated_check_sum'].hist(alpha=.5,label='good',bins=40)\n",
      "plt.legend()"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 203,
       "text": [
        "<matplotlib.legend.Legend at 0x111e38f50>"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x1100c8950>"
       ]
      }
     ],
     "prompt_number": 203
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Concatenate the info into a big pile!\n",
      "df = pd.concat([df_bad, df_good], ignore_index=True)\n",
      "df.replace(np.nan, 0, inplace=True)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [],
     "prompt_number": 26
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Boxplots show you the distribution of the data (spread).\n",
      "# http://en.wikipedia.org/wiki/Box_plot\n",
      "\n",
      "# Get some quick summary stats and plot it!\n",
      "df.boxplot('number_of_import_symbols','label')\n",
      "plt.xlabel('bad vs. good files')\n",
      "plt.ylabel('# Import Symbols')\n",
      "plt.title('Comparision of # Import Symbols')\n",
      "plt.suptitle(\"\")"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 27,
       "text": [
        "<matplotlib.text.Text at 0x11089fb10>"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\nAABJRU5ErkJggg==\n",
       "text": [
        "<matplotlib.figure.Figure at 0x110b0f990>"
       ]
      }
     ],
     "prompt_number": 27
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Get some quick summary stats and plot it!\n",
      "df.boxplot('number_of_sections','label')\n",
      "plt.xlabel('bad vs. good files')\n",
      "plt.ylabel('Num Sections')\n",
      "plt.title('Comparision of Number of Sections')\n",
      "plt.suptitle(\"\")"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 28,
       "text": [
        "<matplotlib.text.Text at 0x1108a3850>"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x1109d4d50>"
       ]
      }
     ],
     "prompt_number": 28
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Split the classes up so we can set colors, size, labels\n",
      "cond = df['label'] == 'good'\n",
      "good = df[cond]\n",
      "bad  = df[~cond]\n",
      "plt.scatter(good['number_of_import_symbols'], good['number_of_sections'], \n",
      "            s=140, c='#aaaaff', label='Good', alpha=.4)\n",
      "plt.scatter(bad['number_of_import_symbols'], bad['number_of_sections'], \n",
      "            s=40, c='r', label='Bad', alpha=.5)\n",
      "plt.legend()\n",
      "plt.xlabel('Import Symbols')\n",
      "plt.ylabel('Num Sections')"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 29,
       "text": [
        "<matplotlib.text.Text at 0x110914f50>"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\nAAAAAElFTkSuQmCC\n",
       "text": [
        "<matplotlib.figure.Figure at 0x1108206d0>"
       ]
      }
     ],
     "prompt_number": 29
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "<div style=\"float: left; padding: 0px 20px 0px 20px;\"><img src=\"files/images/transformers.png\" width=\"300px\"></div>\n",
      "# Data Transformation: \n",
      "** Going from a Pandas DataFrame to an X Matrix and a y vector so we can utilize all of the great scikit-learn algorithms. **"
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# In preparation for using scikit learn we're just going to use\n",
      "# some handles that help take us from pandas land to scikit land\n",
      "\n",
      "# List of feature vectors (scikit learn uses 'X' for the matrix of feature vectors)\n",
      "X = df.as_matrix(['number_of_import_symbols', 'number_of_sections'])\n",
      "\n",
      "# Labels (scikit learn uses 'y' for classification labels)\n",
      "y = np.array(df['label'].tolist())"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [],
     "prompt_number": 131
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Random Forest is a popular ensemble machine learning classifier.\n",
      "# http://scikit-learn.org/dev/modules/generated/sklearn.ensemble.RandomForestClassifier.html\n",
      "#\n",
      "import sklearn.ensemble\n",
      "clf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, compute_importances=True)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [],
     "prompt_number": 132
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Now we can use scikit learn's cross validation to assess predictive performance.\n",
      "scores = sklearn.cross_validation.cross_val_score(clf, X, y, cv=5, n_jobs=4)\n",
      "print scores"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "[ 0.8         0.75        0.7         0.65        0.78947368]\n"
       ]
      }
     ],
     "prompt_number": 133
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Typically you train/test on an 80% / 20%  split meaning you train on 80%\n",
      "# of the data and you test against the remaining 20%. In the case of this\n",
      "# exercise we have so FEW samples (50 good/50 bad) that if were going\n",
      "# to play around with predictive performance it's more meaningful\n",
      "# to train on 60% of the data and test against the remaining 40%.\n",
      "\n",
      "my_seed = 123\n",
      "my_tsize = .4 # 40%\n",
      "from sklearn.cross_validation import train_test_split\n",
      "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=my_tsize, random_state=my_seed)\n",
      "clf.fit(X_train, y_train)\n",
      "y_pred = clf.predict(X_test)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "fragment"
      }
     },
     "outputs": [],
     "prompt_number": 134
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Now plot the results of the 60/40 split in a confusion matrix\n",
      "from sklearn.metrics import confusion_matrix\n",
      "labels = ['good', 'bad']\n",
      "cm = confusion_matrix(y_test, y_pred, labels)\n",
      "plot_cm(cm, labels)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Confusion Matrix Stats\n",
        "good/good: 72.73% (16/22)\n",
        "good/bad: 27.27% (6/22)\n",
        "bad/good: 33.33% (6/18)\n",
        "bad/bad: 66.67% (12/18)\n"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x110c8f910>"
       ]
      }
     ],
     "prompt_number": 135
    },
    {
     "cell_type": "markdown",
     "metadata": {},
     "source": [
      "## Features, predictive performance and 'knobs'\n",
      "Here we going to explore some of the ways you can adjust the 'knobs' associated with either the feature input into your ML algorithm or the prediction probability methods that many classes in scikit-learn have."
     ]
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Okay now try putting in ALL the features (except the label, which would be cheating :)\n",
      "no_label = list(df.columns.values)\n",
      "no_label.remove('label')\n",
      "X = df.as_matrix(no_label)\n",
      "\n",
      "# 60/40 Split for predictive test\n",
      "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=my_tsize, random_state=my_seed)\n",
      "clf.fit(X_train, y_train)\n",
      "y_pred = clf.predict(X_test)\n",
      "cm = confusion_matrix(y_test, y_pred, labels)\n",
      "plot_cm(cm, labels)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Confusion Matrix Stats\n",
        "good/good: 95.45% (21/22)\n",
        "good/bad: 4.55% (1/22)\n",
        "bad/good: 5.56% (1/18)\n",
        "bad/bad: 94.44% (17/18)\n"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x1108def90>"
       ]
      }
     ],
     "prompt_number": 140
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Feature Selection\n",
      "# Which features best deferentiated the two classes?\n",
      "# Here we're going to grab the feature_importances from the classifier itself, \n",
      "# you can also use a Chi Squared Test sklearn.feature_selection.SelectKBest(chi2)\n",
      "importances = zip(no_label, clf.feature_importances_)\n",
      "importances.sort(key=lambda k:k[1], reverse=True)\n",
      "importances[:10]"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "metadata": {},
       "output_type": "pyout",
       "prompt_number": 141,
       "text": [
        "[('compile_date', 0.087118104042058525),\n",
        " ('pe_majorlink', 0.059725488127989876),\n",
        " (u'sec_rawptr_reloc', 0.059172331241524503),\n",
        " ('debug_size', 0.036744505259105907),\n",
        " (u'sec_vasize_reloc', 0.035061138659616312),\n",
        " ('datadir_IMAGE_DIRECTORY_ENTRY_RESOURCE_size', 0.033765884515823991),\n",
        " (u'sec_rawptr_rdata', 0.033184786235755895),\n",
        " ('datadir_IMAGE_DIRECTORY_ENTRY_IAT_size', 0.03261159140279881),\n",
        " ('pe_char', 0.030081949321901769),\n",
        " ('sec_rawsize_text', 0.028226654611623891)]"
       ]
      }
     ],
     "prompt_number": 141
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Produce an X matrix with only the most important featuers\n",
      "X = df.as_matrix([item[0] for item in importances[:10]])\n",
      "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=my_tsize, random_state=my_seed)\n",
      "clf.fit(X_train, y_train)\n",
      "y_pred = clf.predict(X_test)\n",
      "cm = confusion_matrix(y_test, y_pred, labels)\n",
      "plot_cm(cm, labels)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Confusion Matrix Stats\n",
        "good/good: 95.45% (21/22)\n",
        "good/bad: 4.55% (1/22)\n",
        "bad/good: 0.00% (0/18)\n",
        "bad/bad: 100.00% (18/18)\n"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x110913c10>"
       ]
      }
     ],
     "prompt_number": 142
    },
    {
     "cell_type": "code",
     "collapsed": false,
     "input": [
      "# Compute the predition probabilities and use them to mimimize our false positives\n",
      "# Note: This is simply a trade off, it means we'll miss a few of the malicious\n",
      "# ones but typically false alarms are a death blow to any new 'fancy stuff' so\n",
      "# we definitely want to mimimize the false alarms.\n",
      "y_probs = clf.predict_proba(X_test)[:,0]\n",
      "thres = .8 # This can be set to whatever you'd like\n",
      "y_pred[y_probs<thres] = 'good'\n",
      "y_pred[y_probs>=thres] = 'bad'\n",
      "cm = confusion_matrix(y_test, y_pred, labels)\n",
      "plot_cm(cm, labels)"
     ],
     "language": "python",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "outputs": [
      {
       "output_type": "stream",
       "stream": "stdout",
       "text": [
        "Confusion Matrix Stats\n",
        "good/good: 100.00% (22/22)\n",
        "good/bad: 0.00% (0/22)\n",
        "bad/good: 16.67% (3/18)\n",
        "bad/bad: 83.33% (15/18)\n"
       ]
      },
      {
       "metadata": {},
       "output_type": "display_data",
       "png": "...\n",
       "text": [
        "<matplotlib.figure.Figure at 0x110c868d0>"
       ]
      }
     ],
     "prompt_number": 143
    },
    {
     "cell_type": "markdown",
     "metadata": {
      "slideshow": {
       "slide_type": "slide"
      }
     },
     "source": [
      "## Conclusions:\n",
      "<font size=4>\n",
      "The combination of IPython, Pandas and Scikit Learn let us pull in PE files, extract features, plot them, understand them and slap them with some machine learning!\n",
      "<br><br>\n",
      "As mentioned in the disclaimer, the biggest issue with this particular exercise is the small number of samples. \n",
      "<br><br>\n",
      "There are some really great machine learning resources that cover this material on a deeper and more formal level. In particular we highly recommend the work and presentations of Olivier Grisel at INRIA Saclay. http://ogrisel.com/\n",
      "</font>"
     ]
    }
   ],
   "metadata": {}
  }
 ]
}
